In [1]:
import findspark
import numpy as np
import pandas as pd
import os
import re
findspark.init()
findspark.find()

'D:\\Programs\\spark-3.5.0-bin-hadoop3'

In [2]:
def text_split(text):
    delimiter = r'[ \n\t,\.!\?:;\"\(\)\[\]]+'
    return re.split(delimiter, text.lower())

In [3]:
import pyspark
sc = pyspark.SparkContext("local", "lab1")

In [4]:
rdd_alcohol = sc.textFile("text_docs/Alcohol.txt")
rdd_cannabis = sc.textFile("text_docs/Cannabis.txt")
rdd_cocaine = sc.textFile("text_docs/Cocaine.txt")
rdd_heroin = sc.textFile("text_docs/Heroin.txt")
rdd_nicotine = sc.textFile("text_docs/Nicotine.txt")

In [5]:
def word_count(rdd):
    words = rdd.flatMap(text_split).countByValue()
    word_counts = dict(words)
    del word_counts['']
    # Повертає звичайний dict
    return word_counts

# Гарненько друкує n найрозповсюдженіших слів зі словника dict_words
def print_top_n(dict_words, n=30):
    words_sorted = sorted(dict_words.items(), key=lambda x: x[1], reverse=True)
    words_top_n = pd.DataFrame(words_sorted[:n], columns=["word", "count"])
    print(words_top_n)

In [6]:
wordcount_alcohol = word_count(rdd_alcohol)
wordcount_cannabis = word_count(rdd_cannabis)
wordcount_cocaine = word_count(rdd_cocaine)
wordcount_heroin = word_count(rdd_heroin)
wordcount_nicotine = word_count(rdd_nicotine)

In [7]:
print_top_n(wordcount_alcohol, n=30)

              word  count
0               of    305
1              the    293
2              and    248
3               in    198
4          alcohol    180
5               to    149
6               is    133
7          ethanol    125
8                a    111
9               as     81
10            with     68
11             are     66
12              or     58
13            that     52
14         effects     49
15              by     48
16              it     40
17             for     40
18             can     40
19     consumption     39
20            also     33
21              at     33
22             has     32
23              be     31
24              on     29
25        drinking     28
26            such     27
27            have     27
28  concentrations     27
29            more     26


In [8]:
rdd_drugs = sc.wholeTextFiles("text_docs/*")

In [9]:
def tokenize(text, doc_id):
    words = text_split(text)
    term_locations = {}

    for position, word in enumerate(words):
        if word not in term_locations:
            term_locations[word] = [os.path.basename(doc_id), []]
        term_locations[word][1].append(position)

    return [(word, locations) for word, locations in term_locations.items()]

#RDD має бути саме у вигляді [(назва1, вміст1), ...], як наприклад від wholeTextFiles
def inverted_index(rdd):
    inverted_index_raw = rdd.flatMap(lambda x: tokenize(x[1], x[0])).groupByKey().mapValues(list)
    inverted_index = inverted_index_raw.collectAsMap()
    del inverted_index[""]
    # повертає звичайний dict
    return inverted_index

def print_invertedindex(arr):
    for row in arr:
        print(f"{row[0]}: {row[1]}")

In [10]:
inv_index = inverted_index(rdd_drugs)

In [11]:
print_invertedindex(inv_index["depression"])

Alcohol.txt: [73, 2340, 2438, 2917, 5028, 5216]
Cannabis.txt: [1782, 2526, 2594, 3226]
Cocaine.txt: [399]
Heroin.txt: [183, 1973, 2027, 3395]
Nicotine.txt: [605]
